In [ ]:
import pandas as pd
from scipy import stats

from pathlib import Path
import pickle
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import r2_score

from rfpimp import permutation_importances

from datetime import datetime

import numpy as np

from UTILS import utils

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
output_dir = Path.cwd().joinpath('OUTPUT')
column_dir = output_dir.joinpath('COLUMNS')
config_dir = Path.cwd().joinpath('CONFIG')
model_dir = output_dir.joinpath('MODELS')
image_dir = output_dir.joinpath('IMAGES')

# Load Data

In [ ]:
data_dir = Path.cwd().joinpath('OUTPUT').joinpath('df_merged')
with open(data_dir, 'rb') as infile:
    df_merged = pickle.load(infile)

In [ ]:
df_merged.shape

## Add the Features

Exclude the columns that are not included, according to `/CONFIG/ml_columns.csv`.

In [ ]:
ml_columns_df = utils.load_data(
    config_dir,
    'ml_columns.csv',
    )

In [ ]:
ml_columns = ml_columns_df.query('use == 1')['columns'].values

In [ ]:
df_features = df_merged.copy(deep=True)

for filename in column_dir.iterdir():
    if str(filename.stem) in ml_columns:
        print(f'merging {filename.stem}')
        df_features = utils.add_column(df=df_features,
                                       data_dir=column_dir,
                                       filename=str(filename.stem)
                                      )

# 'student_comment_pos_tags' is actually a dataframe, not a series. So the column name is not 'student_comment_pos_tags'.        
df_features = utils.add_column(df=df_features,
                               data_dir=column_dir,
                               filename='student_comment_pos_tags'
                              )

In [ ]:
df_features = df_features[ml_columns]

In [ ]:
df_features.shape

# Setting Up the Data

## Subset the Data and Set the Data Type

1. Keep only the columns that would be usable by the machine learning algorithms; the mapping is stored in `/CONFIG/ml_columns.csv`.
2. Keep only the rows that have a student rating.
3. Configure the data types according to `/CONFIG/mapping_column_types_extended.csv`.

In [ ]:
df_subset = df_features[df_features.student_rating_numeric.notnull()]

In [ ]:
sorted(df_subset.columns)

## Setting the Data Types

In [ ]:
mapping_column_types_extended = utils.load_data(
    config_dir,
    'mapping_column_types_extended.csv'
    ).set_index('columns')

mapping_column_types_extended.head()

In [ ]:
df_subset.qualifications

In [ ]:
df_subset = (df_subset
             .apply(lambda x: utils.map_column_dtype(x, mapping_column_types_extended))
            )  

## Imputing Missing Data

Imputation of missing values is required because the algorithms in `sklearn` does not handle missing values properly. A simple imputation scheme is employed whereby a new category "missing" is imputed for categorical variables and a 0 is imputed for numeric variables.

In [ ]:
df_imputed = utils.simple_impute(df_subset)

## Convert the Categorical Variable to Dummy Variables

Some algorithms in the `sklearn` package cannot deal with string-valued categorical variables. Therefore we will now convert such variables into one dummy variables.

In [ ]:
cat_columns = df_imputed.select_dtypes(include='category').columns.tolist()
cat_columns

In [ ]:
df_dummies = pd.get_dummies(df_imputed,
                            drop_first=True)
df_dummies.head()

In [ ]:
utils.save_object(
    df_dummies,
    'df_dummies',
    output_dir)

## Split Into X and y

The `y` is a column that contains the values that we want to predict, i.e. the `student_rating`. The `X` is a set of columns that are used to predict the `y`, e.g. waiting time, tutor age, etc

In [ ]:
X, y = df_dummies.iloc[:, 1:], df_dummies.iloc[:, 0]

In [ ]:
X.shape

In [ ]:
y.head()

## Train Test Split

The data is split such that the proportions of the values of target variable, i.e. `student_rating_fixed` is maintained after the split. This is called stratification and has been shown to produce a better results. In the `test_train_split` function, the parameter is `stratify`.

In [ ]:
utils.calc_percentage_counts(
    df_dummies.student_rating_numeric,
    )

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=1,
    stratify=y)

In [ ]:
utils.calc_percentage_counts(
    y_train,
    )

From the above table, it can be seen that the counts are about 70% of the previous table, however, the proportions are maintained. 

# Random Forest Regression

## Random Forest Regression Training

The training uses `RandomizedSearchCV`, which searches a random sample of the hyperparameters. The score used is the [`neg_mean_squared_error`](neg_mean_squared_error) negative mean square error, which is scale dependent and is directly interpretable, the negation is purely because the `RandomizedSearchCV` maximises a criteria, rather than minimises it, so the smaller the negative number the better performing the algorithm is.

In [ ]:
start_time = datetime.now()
print(f'Training started at {start_time}.')

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

regressor = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 5, 
                               cv = 5, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)
# Fit the random search model
rf_random.fit(X_test, y_test)

end_time = datetime.now()
print(f'Training completed at {end_time}.')
run_time = end_time - start_time
print(f'Runtime was {run_time}.')

In [ ]:
rf_random = utils.load_data(
    model_dir,
    'rf_random')

In [ ]:
rf_random.best_params_

## Random Forest Grid Search

In [ ]:
start_time = datetime.now()
print(f'Training started at {start_time}.')

# Number of trees in random forest
n_estimators = [550, 650]
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [80, 100]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [9, 11]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4, 5]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
grid = {'n_estimators': n_estimators,
        'max_features': max_features,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'bootstrap': bootstrap}

regressor = RandomForestRegressor()

rf_grid = GridSearchCV(estimator = regressor, 
                       param_grid = grid, 
                       cv = 5, 
                       verbose=2, 
                       n_jobs = -1)
# Fit the random search model
rf_grid.fit(X_train, y_train)

end_time = datetime.now()
print(f'Training completed at {end_time}.')
run_time = end_time - start_time
print(f'Runtime was {run_time}.')

In [ ]:
utils.save_object(
    rf_grid,
    'rf_grid',
    model_dir)

In [ ]:
rf_grid.best_params_

## Random Forest RMSE

In [ ]:
preds = rf_grid.best_estimator_.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print(f'RMSE: {rmse}')

In [ ]:
rf_grid = utils.load_data(
    model_dir,
    'rf_grid'
)

## Random Forest Variable Importance

In [ ]:
rf_grid = utils.load_data(
    model_dir,
    'rf_grid'
    )

In [ ]:
feature_importances = pd.DataFrame(rf_grid.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
feature_importances.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20,30))

sns.barplot(x='importance',
            y=feature_importances.index,
            data=feature_importances,
           )

# Saving
model = 'rf'
extension = 'png'
filename = f'variable_importance_{model}.{extension}'
filepath = image_dir.joinpath(filename)
plt.tight_layout()
plt.savefig(filepath)

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))

sns.barplot(x='importance',
            y=feature_importances.head().index,
            data=feature_importances.head(),
           )

# Saving
model = 'rf'
extension = 'png'
filename = f'variable_importance_{model}_top_5.{extension}'
filepath = image_dir.joinpath(filename)
plt.tight_layout()
plt.savefig(filepath)

# XGBoost

## Regression

In [ ]:
start_time = datetime.now()
print(f'Training started at {start_time}.')

regressor = xgb.XGBRegressor(objective='reg:squarederror')

params = {
    'n_estimators': stats.randint(3, 40),
    'max_depth': stats.randint(3, 40),
    'learning_rate': stats.uniform(0.05, 0.4),
    'colsample_bytree': stats.beta(10, 1),
    'subsample': stats.beta(10, 1),
    'gamma': stats.uniform(0, 10),
    'reg_alpha': stats.expon(0, 50),
    'min_child_weight': stats.expon(0, 50),
}
cv_results = RandomizedSearchCV(estimator = regressor, 
                                param_distributions = params, 
                                n_iter = 20, 
                                cv = 5, 
                                verbose=2, 
                                random_state=42, 
                                n_jobs = -1
                               )

cv_results.fit(X_train, y_train)

end_time = datetime.now()
print(f'Training completed at {end_time}.')

run_time = end_time - start_time

print(f'Runtime was {run_time}.')

In [ ]:
utils.save_object(cv_results, 
                  'xgb_regressor_randomizedsearchcv', 
                  model_dir)

### RMSE

In [ ]:
preds = cv_results.best_estimator_.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print(f'RMSE: {rmse}')

In [ ]:
cv_results.best_params_

## XGBoost Regression Feature Importance

## Full Number of Features

In [ ]:
fig, ax = plt.subplots(figsize=(20,25))
xgb.plot_importance(xgb_regressor_randomizedsearchcv.best_estimator_,
                    ax=ax)

# Saving
model = 'xgb'
extension = 'png'
filename = f'variable_importance_{model}.{extension}'
filepath = image_dir.joinpath(filename)
plt.tight_layout()
plt.savefig(filepath)

## Top 10 Features

In [ ]:
figsize = (10,3)
max_num_features = 10
fig, ax = plt.subplots(figsize=figsize)

plt.tight_layout()

xgb.plot_importance(xgb_regressor_randomizedsearchcv.best_estimator_,
                    max_num_features=max_num_features,
                    ax=ax)

# Saving
model = 'xgb'
extension = 'png'

filename = f'~variable_importance_{model}_top_{max_num_features}.{extension}'
filepath = image_dir.joinpath(filename)
plt.tight_layout()
plt.savefig(filepath)

# Support Vector Regression

## PCA (Principle Components Analysis)

SVM is known to take a long time, especially if there are a lot of features, so it would be more feasible to apply some dimensionality reduction techniques to reduce the 100 columns to something more manageble. To that end PCA is applied.

In [ ]:
pca = PCA()
pca.fit(X_train)

plt.figure(1, figsize=(20, 10))
plt.clf()
plt.axes([.2, .2, .7, .7])
plt.plot(pca.explained_variance_, linewidth=2)
plt.axis('tight')
plt.xlabel('n_components')
plt.ylabel('explained_variance_')

The elbow occurs at very low n_components. Which means that the data set is highly correlated.

In [ ]:
start_time = datetime.now()
print(f'SVR started at {start_time}.')

pipe_svr = make_pipeline(PCA(n_components=5),
                         StandardScaler(),
                         SVR())
pipe_svr.fit(X_train, y_train)

end_time = datetime.now()
print(f'Training completed at {end_time}.')

run_time = end_time - start_time

print(f'Runtime was {run_time}.')

In [ ]:
X_train.iloc[:, :50].shape

10,000 rows → 15 seconds
20,000 rows → 3 minutes
20,000 rows with 50 columns → 1:24
20,000 rows with 10 columns → 41 seconds
20,000 rows with 5 columns → 16 seconds
100,000 rows with 5 columns → 16 minutes

In [ ]:
X_train_small = X_train.sample(100000).iloc[:, :5]
y_train_small = y_train.sample(100000)

start_time = datetime.now()
print(f'SVR started at {start_time}.')

pipe_svr = make_pipeline(PCA(),
                         StandardScaler(),
                         SVR(),

params = {
    'pca__n_components': [3, 4, 5, 6],
    'svr__kernel': ['rbf', 'poly'],
    'svr__C': [1, 5, 10, 20, 40],
    'svr__degree': [2, 3, 4, 5, 6]
}

cv_results = RandomizedSearchCV(estimator=pipe_svr,
                                param_distributions=tributions=params,
                                n_iter=12,
                                cv=5,
                                verbose=2,
                                random_state=42,
                                n_jobs=-1
                               )

cv_results.fit(X_train, y_train)

end_time = datetime.now()
print(f'Training completed at {end_time}.')

run_time = end_time - start_time

print(f'Runtime was {run_time}.')

# Ensemble

## `VotingRegressor` pre-fitted

After tuning the hyperparameters of the two models using `RandomizedSearchCV`, the results can be combined to potentially create a better performing regressor. This is done by using the `VotingRegressor` module.

In [ ]:
start_time = datetime.now()
print(f'SVR started at {start_time}.')

vr = VotingRegressor([('lr', rf_grid.best_estimator_), 
                      ('rf', xgb_regressor_randomizedsearchcv.best_estimator_)])

vr.fit(X_train, y_train)

end_time = datetime.now()
print(f'Training completed at {end_time}.')

run_time = end_time - start_time

print(f'Runtime was {run_time}.')

In [ ]:
utils.save_object(
    vr,
    'vr_xgb_rf_prefitted',
    model_dir
    )

### `VotingRegressor` pre-fitted RMSE

In [ ]:
preds = vr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print(f'RMSE: {rmse}')

## `VotingRegressor` Unfitted Using `RandomizedSearchCV` `best_params_`

Unfitted models but using the best parameters obtained from the `RandomizedSearchCV`.

In [ ]:
start_time = datetime.now()
print(f'SVR started at {start_time}.')

vr = VotingRegressor([('rf', RandomForestRegressor(n_estimators=600,
                                                   min_samples_split=10,
                                                   min_samples_leaf=4,
                                                   max_features='sqrt',
                                                   max_depth=90,
                                                   bootstrap=False)), 
                      ('xgb', xgb.XGBRegressor(colsample_bytree=0.9252155845351104,
                                               gamma=1.5601864044243652,
                                               learning_rate=0.11239780813448107,
                                               max_depth=13,
                                               min_child_weight=30.73980825409684,
                                               n_estimators=38,
                                               reg_alpha=7.708098373328053,
                                               subsample=0.9937572296628479)
                      )])

vr.fit(X_train, y_train)

end_time = datetime.now()
print(f'Training completed at {end_time}.')

run_time = end_time - start_time

print(f'Runtime was {run_time}.')

In [ ]:
utils.save_object(
    vr,
    'vr_xgb_rf_unfitted',
    model_dir
    )

### `VotingRegressor` un-fitted RMSE

In [ ]:
preds = vr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print(f'RMSE: {rmse}')

## `VotingRegressor` Unfitted

Unfitted fresh estimators.

In [ ]:
start_time = datetime.now()
print(f'SVR started at {start_time}.')

vr = VotingRegressor([('rf', RandomForestRegressor()), 
                      ('xgb', xgb.XGBRegressor()
                      )])

vr.fit(X_train, y_train)

end_time = datetime.now()
print(f'Training completed at {end_time}.')

run_time = end_time - start_time

print(f'Runtime was {run_time}.')

In [ ]:
utils.save_object(
    vr,
    'vr_xgb_rf_unfitted_fresh',
    model_dir
    )

### `VotingRegressor` un-fitted RMSE

In [ ]:
preds = vr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print(f'RMSE: {rmse}')